## Dependencies

In [1]:
!pip install gensim

In [2]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [3]:
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout, Masking, Bidirectional, Multiply, Lambda
from tensorflow.keras import Model, Input
import gensim as gs
from tensorflow.keras.models import Model
import ast
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import ReduceLROnPlateau

In [4]:
# #! check if tensorflow is using GPU
# gpus = tf.config.list_physical_devices('GPU')
# print(tf.__version__)
# if gpus:
#     print(f"TensorFlow is using {len(gpus)} GPU(s).")
#     for gpu in gpus:
#         print(f"GPU: {gpu.name}")
# else:
#     print("TensorFlow is not using any GPUs.")

In [5]:
def load_data_labels(data_path, labels_path):
    with open(data_path, 'r') as f:
        data = [ast.literal_eval(line.strip()) for line in f]
    with open(labels_path, 'r') as f:
        labels = [ast.literal_eval(line.strip()) for line in f]
    return data, labels

In [6]:
# data, labels = load_data_labels('training_data_processed.txt', 'train_order_category_labels.txt')
data, labels = load_data_labels('train_data_order_details.txt', 'train_labels_order_details.txt')
_,Xmodel1=load_data_labels('trian_data_order_category.txt','train_labels_order_category.txt')
# data2, labels2 = load_data_labels('synthetic_orders.txt', 'synthetic_labels.txt')
# data.extend(data2)
# labels.extend(labels2)
print(len(data))
print(len(labels))
print(data[:5])
print(labels[:5])
print(len(Xmodel1))
print(Xmodel1[:5])

51000
51000
[['i', 'like', 'one', 'pizza', 'with', 'red', 'onion', 'fry', 'onion', 'and', 'mozarella', 'without', 'thin', 'crust'], ['i', 'like', 'one', 'pizza', 'with', 'anchovy', 'caramelize', 'red', 'onion', 'and', 'roast', 'green', 'pepper', 'without', 'thin', 'crust'], ['i', 'like', 'one', 'pizza', 'with', 'applewood', 'bacon', 'grill', 'pineapple', 'and', 'shrimp', 'without', 'thin', 'crust'], ['i', 'like', 'one', 'pizza', 'with', 'pesto', 'sauce', 'roast', 'pepper', 'and', 'peppperonis', 'without', 'thin', 'crust'], ['i', 'like', 'one', 'pizza', 'with', 'peperronni', 'spicy', 'red', 'sauce', 'and', 'mushroom', 'without', 'thin', 'crust']]
[[0, 0, 1, 0, 0, 4, 4, 4, 4, 0, 4, 0, 7, 7], [0, 0, 1, 0, 0, 4, 4, 4, 4, 0, 4, 4, 4, 0, 7, 7], [0, 0, 1, 0, 0, 4, 4, 4, 4, 0, 4, 0, 7, 7], [0, 0, 1, 0, 0, 4, 4, 4, 4, 0, 4, 0, 7, 7], [0, 0, 1, 0, 0, 4, 4, 4, 4, 0, 4, 0, 7, 7]]
51000
[[2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 2, 0, 0, 0, 0,

In [7]:
# dev_data, dev_labels = load_data_labels('dev_data_processed.txt', 'dev_order_category_labels.txt')
dev_data, dev_labels = load_data_labels('dev_data_order_details.txt', 'dev_labels_order_details.txt')
_,devXmodel1=load_data_labels('dev_data_order_category.txt','dev_labels_order_category.txt')
print(len(devXmodel1))
print(devXmodel1[:5])

847
[[2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0], [2, 2, 2, 2, 0, 0, 0, 0, 0, 0], [2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


In [8]:
pretrained_model = gs.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [9]:
#! get v Aand replace unknown words with unk token
def process_sentence(sentence, model):
    for i, word in enumerate(sentence):
        if word not in model:
            sentence[i] = 'unk'
    return sentence

data = [process_sentence(sentence, pretrained_model) for sentence in data]
print(data[:5])
vocab=set()
for sentence in data:
    vocab.update(sentence)
#! get word index for each word in vocab
word2idx = {word: idx for idx, word in enumerate(vocab)}

[['i', 'like', 'one', 'pizza', 'with', 'red', 'onion', 'fry', 'onion', 'unk', 'mozarella', 'without', 'thin', 'crust'], ['i', 'like', 'one', 'pizza', 'with', 'anchovy', 'caramelize', 'red', 'onion', 'unk', 'roast', 'green', 'pepper', 'without', 'thin', 'crust'], ['i', 'like', 'one', 'pizza', 'with', 'applewood', 'bacon', 'grill', 'pineapple', 'unk', 'shrimp', 'without', 'thin', 'crust'], ['i', 'like', 'one', 'pizza', 'with', 'pesto', 'sauce', 'roast', 'pepper', 'unk', 'unk', 'without', 'thin', 'crust'], ['i', 'like', 'one', 'pizza', 'with', 'unk', 'spicy', 'red', 'sauce', 'unk', 'mushroom', 'without', 'thin', 'crust']]


In [10]:
embedding_dim=300
input_dim=len(vocab)
output_dim=11
max_length=100

In [11]:
for tokens in dev_data:
    for i,word in enumerate(tokens):
        if word not in vocab:
            tokens[i] = 'unk'
X_d=[[word2idx[word] for word in sentence] for sentence in dev_data]
X_d=pad_sequences(X_d, maxlen=max_length, padding='post', value=-1)
X_categories_d = [[category for category in sentence_categories] for sentence_categories in devXmodel1]
X_categories_d = pad_sequences(X_categories_d, maxlen=max_length, padding='post', value=2)  # Default to "NEITHER" = 2
Y_d=pad_sequences(dev_labels, maxlen=max_length, padding='post', value=0)


In [12]:
#! get embeddings matrix
def get_embeddings_matrix(model, vocab):
    vocab_size = len(vocab)
    embedding_dim = model.vector_size
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    for i, word in enumerate(vocab):
        if word in model:
            embedding_matrix[i] = model[word]
    return embedding_matrix
embedding_matrix = get_embeddings_matrix(pretrained_model, vocab)

In [13]:
#! replace words with their index in vocab and pad sentences
X=[[word2idx[word] for word in sentence] for sentence in data]
X=pad_sequences(X, maxlen=max_length, padding='post', value=-1)
X_categories = [[category for category in sentence_categories] for sentence_categories in Xmodel1]
X_categories = pad_sequences(X_categories, maxlen=max_length, padding='post', value=2)  # Default to "NEITHER" = 2
Y=pad_sequences(labels, maxlen=max_length, padding='post', value=0)

In [14]:
# input_seq = Input(shape=(None,), dtype=tf.int32)
# masked_input = Masking(mask_value=-1)(input_seq)

# embedding = Embedding(input_dim=input_dim, output_dim=embedding_dim, weights=[embedding_matrix], trainable=True, mask_zero=False)(masked_input)

# lstm_output = Bidirectional(LSTM(32, return_sequences=True))(embedding)

# query = LSTM(32, return_sequences=True)(lstm_output)  
# key = LSTM(32, return_sequences=True)(lstm_output)    
# value = LSTM(32, return_sequences=True)(lstm_output)  

# attention_output = Attention(use_scale=True, causal=False)([query, key, value])

# dropout_output = Dropout(0.8)(attention_output)
# output = Dense(output_dim, activation='softmax')(dropout_output)

# model = Model(inputs=input_seq, outputs=output)

# optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

# model.compile(
#     optimizer=optimizer,
#     loss='sparse_categorical_crossentropy',
#     metrics=['accuracy']
# )

# model.summary()

# lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
#     monitor='val_loss',
#     factor=0.5,
#     patience=5,
#     min_lr=1e-9,
#     verbose=1
# )

In [15]:
# model.fit(
#     X, 
#     Y, 
#     validation_data=(X_d, Y_d),  # Add validation data
#     callbacks=[lr_scheduler], 
#     epochs=25,
#     batch_size=1024
# )


In [16]:


# Define category mask function
def create_category_mask(categories):
    # Updated mask for 11 output dimensions (aligned with the output_dim)
    category_to_mask = tf.constant([
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],  # PIZZA
        [1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1],  # DRINK
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]   # NEITHER
    ], dtype=tf.float32)  # Shape: (3, 11) for output_dim of 11

    # Replace padding (-1) with NEITHER (category index 2)
    categories = tf.where(categories == -1, 2, categories)

    # Create the mask: gather the appropriate row based on the category index
    mask = tf.gather(category_to_mask, categories)  # Shape: (batch_size, max_length, 11)
    return mask

# Define token input and category input
input_tokens = Input(shape=(max_length,), dtype='int32', name='tokens')  # Token indices
input_categories = Input(shape=(max_length,), dtype='int32', name='categories')  # Category predictions

# Embedding layer for tokens (with a pre-trained embedding matrix)
x = Masking(mask_value=-1)(input_tokens)  # Mask the padding (-1)
x = Embedding(input_dim=len(vocab), output_dim=embedding_dim, 
              weights=[embedding_matrix], trainable=True)(x)

# Embedding layer for categories (fixed size of 3 categories: PIZZA, DRINK, NEITHER)
category_embedding = Embedding(input_dim=3, output_dim=8, trainable=True)(input_categories)

# Concatenate token embeddings and category embeddings
x = tf.concat([x, category_embedding], axis=-1)  # Concatenate along the last dimension

# BiLSTM and Dropout layer
x = Bidirectional(LSTM(64, return_sequences=True))(x)  # BiLSTM to capture sequential dependencies
x = Dropout(0.6)(x)  # Dropout to avoid overfitting

# Dense layer for logits (predictions) without softmax yet
logits = Dense(output_dim, activation=None)(x)  # Output shape: (batch_size, max_length, 11)

# Apply category-specific masking (to set certain logits to zero based on category)
mask = Lambda(create_category_mask)(input_categories)  # Apply the category mask
masked_logits = Multiply()([logits, mask])  # Element-wise multiplication of logits and mask

# Final softmax activation for probabilities
output = tf.keras.activations.softmax(masked_logits, axis=-1)  # Apply softmax across the last dimension

# Build and compile the model
model = Model(inputs=[input_tokens, input_categories], outputs=output)

# Compile the model with Adam optimizer, sparse categorical crossentropy loss, and accuracy as a metric
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',      # The metric to monitor (you can use 'accuracy' or another metric)
    factor=0.5,              # Factor by which the learning rate will be reduced
    patience=5,              # Number of epochs with no improvement after which learning rate will be reduced
    min_lr=1e-9,             # Minimum value for the learning rate
    verbose=1                # Print a message when the learning rate is reduced
)

# Train the model
model.fit(
    [X, X_categories],  # Token sequences (X) and category predictions (X_categories)
    Y,                  # Labels for token classification (Y)
    validation_data=([X_d, X_categories_d], Y_d),  # Validation data
    epochs=25,  # Number of epochs
    batch_size=512,  # Batch size
    callbacks=[lr_scheduler]  # Learning rate scheduler
)


Epoch 1/25
100/100 [==============================] - 14s 54ms/step - loss: 2.2165 - accuracy: 0.9769 - val_loss: 2.1804 - val_accuracy: 0.9978 - lr: 0.0010
Epoch 2/25
100/100 [==============================] - 5s 46ms/step - loss: 2.1397 - accuracy: 0.9990 - val_loss: 2.1737 - val_accuracy: 0.9987 - lr: 0.0010
Epoch 3/25
100/100 [==============================] - 5s 47ms/step - loss: 2.1365 - accuracy: 0.9996 - val_loss: 2.1724 - val_accuracy: 0.9990 - lr: 0.0010
Epoch 4/25
100/100 [==============================] - 5s 46ms/step - loss: 2.1356 - accuracy: 0.9998 - val_loss: 2.1716 - val_accuracy: 0.9993 - lr: 0.0010
Epoch 5/25
100/100 [==============================] - 5s 49ms/step - loss: 2.1352 - accuracy: 0.9999 - val_loss: 2.1711 - val_accuracy: 0.9994 - lr: 0.0010
Epoch 6/25
100/100 [==============================] - 5s 49ms/step - loss: 2.1350 - accuracy: 0.9999 - val_loss: 2.1709 - val_accuracy: 0.9995 - lr: 0.0010
Epoch 7/25
100/100 [==============================] - 5s 50ms/s

In [17]:
# # X was index of token in embedding matrix
# # X -> 

# model.fit(
#     X, 
#     Y, 
#     validation_data=(X_d, Y_d),  # Add validation data
#     callbacks=[lr_scheduler], 
#     epochs=25,
#     batch_size=512
# )

In [18]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 tokens (InputLayer)            [(None, 100)]        0           []                               
                                                                                                  
 masking (Masking)              (None, 100)          0           ['tokens[0][0]']                 
                                                                                                  
 categories (InputLayer)        [(None, 100)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 100, 300)     112800      ['masking[0][0]']                
                                                                                              

## Evaluate model on training data

In [19]:
# Assuming you have your token data in X and category data in X_categories
preds_train = model.predict([X, X_categories])  # Pass both inputs as a list
preds_train = np.argmax(preds_train, axis=-1)  # Get the class with the highest probability


1594/1594 [==============================] - 21s 13ms/step


In [20]:
count = 0
for i in range(len(X)):
    mask=X[i]!=-1
    if np.all(preds_train[i][mask]==Y[i][mask]):
        count+=1
print(f"Accuracy on training data: {count/len(X)}")

Accuracy on training data: 0.9994313725490196


In [21]:
# # Predict on the development set
# preds_dev = Bidirectional_LSTM_model.predict(X)
# preds_dev = np.argmax(preds_dev, axis=-1)

# count = 0  # Count of completely correct sequences
# last_index_error_count = 0  # Count of sequences with only last index error

# for i in range(len(data)):
#     # Get original sequence lengths before padding
#     original_length = len(data[i])
    
#     # Extract predictions and true labels for the original sequence
#     pred_seq = preds_dev[i][:original_length]
#     true_seq = labels[i]  # Original labels are unpadded
    
#     # Check if the sequence is entirely correct
#     if (pred_seq == true_seq).all():
#         count += 1
#     else:
#         # Check if only the last index is incorrect
#         if (pred_seq[:-1] == true_seq[:-1]).all() and (pred_seq[-1] != true_seq[-1]):
#             last_index_error_count += 1
        
#         # Print debug information for mismatches
#         print(f"Index with mismatch: {i}")
#         print(f"Predicted: {pred_seq}")
#         print(f"True:      {true_seq}")
#         print("--------------------------------------------")

# # Print results
# print(f"Sequences with only last index error: {last_index_error_count}")


## Evaluate model on dev data

In [22]:
# dev_data, dev_labels = load_data_labels('dev_data_processed.txt', 'dev_order_category_labels.txt')
# for tokens in dev_data:
#     for i,word in enumerate(tokens):
#         if word not in vocab!= 'a':
#             tokens[i] = 'unk'
# X_d=[[word2idx[word] for word in sentence] for sentence in dev_data]
# X_d=pad_sequences(X_d, maxlen=max_length, padding='post', value=-1)
# Y_d=pad_sequences(dev_labels, maxlen=max_length, padding='post', value=2)

In [23]:
# Predict on the development set
preds_dev = model.predict([X_d,X_categories_d])
preds_dev = np.argmax(preds_dev, axis=-1)

count = 0  # Count of completely correct sequences
last_index_error_count = 0  # Count of sequences with only last index error

for i in range(len(dev_data)):
    # Get original sequence lengths before padding
    original_length = len(dev_data[i])
    
    # Extract predictions and true dev_labels for the original sequence
    pred_seq = preds_dev[i][:original_length]
    true_seq = dev_labels[i]  # Original dev_labels are unpadded
    
    # Check if the sequence is entirely correct
    if (pred_seq == true_seq).all():
        count += 1
    else:
        # Check if only the last index is incorrect
        if (pred_seq[:-1] == true_seq[:-1]).all() and (pred_seq[-1] != true_seq[-1]):
            last_index_error_count += 1
        
        # Print debug information for mismatches
        print(f"Index with mismatch: {i}")
        
        print(f"Predicted: {pred_seq}")
        print(f"True:      {true_seq}")
        print("--------------------------------------------")

# Print results
print(f"Accuracy on dev_data: {count / len(dev_data):.4f}")
print(f"Sequences with only last index error: {last_index_error_count}")


27/27 [==============================] - 0s 15ms/step
Index with mismatch: 163
Predicted: [0 0 0 0 1 8 8 8 1 8 8 0 1 8 8 0 2 2]
True:      [0, 0, 0, 0, 1, 8, 8, 0, 1, 8, 0, 0, 1, 8, 0, 0, 2, 0]
--------------------------------------------
Index with mismatch: 180
Predicted: [0 0 1 2 2 0]
True:      [0, 0, 1, 2, 0, 0]
--------------------------------------------
Index with mismatch: 218
Predicted: [0 0 1 2 1 0 0 0 4 0 4]
True:      [0, 0, 1, 2, 0, 0, 0, 0, 4, 0, 4]
--------------------------------------------
Index with mismatch: 224
Predicted: [0 0 1 2 0 0 0 0 4 0 4 0 0 0 0 0 0 0 0 7 3]
True:      [0, 0, 1, 2, 0, 0, 0, 0, 4, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 7, 7]
--------------------------------------------
Index with mismatch: 228
Predicted: [0 0 0 1 3 3 0 0 5 4]
True:      [0, 0, 0, 1, 0, 3, 0, 0, 5, 4]
--------------------------------------------
Index with mismatch: 230
Predicted: [0 0 0 0 0 1 2 3 3 0 0 4 4 0 0 4 0 1 4 0 4 4 0 1 2 8]
True:      [0, 0, 0, 0, 0, 1, 2, 3, 3, 0, 0, 4, 4, 

In [24]:
# Save in HDF5 format
# model.save("order_details_model.h5")
# model= tf.keras.models.load_model("order_details_model.h5")

In [25]:
test_data,test_labels_order_category=load_data_labels('test_data_order_details.txt','test_labels_order_category.txt')
print(test_data)
print(test_labels_order_category)
for tokens in test_data:
    for i,word in enumerate(tokens):
        if word not in vocab:
            tokens[i] = 'unk'
X_test=[[word2idx[word] for word in sentence] for sentence in test_data]
X_test=pad_sequences(X_test, maxlen=max_length, padding='post', value=-1)
X_categories_test = [[category for category in sentence_categories] for sentence_categories in test_labels_order_category]
X_categories_test = pad_sequences(X_categories_test, maxlen=max_length, padding='post', value=2)

[['want', 'one', 'pizza', 'with', 'pepproni', 'and', 'large', 'size', 'pepproni', 'pizza']]
[[2, 0, 0, 0, 0, 2, 0, 0, 0, 0]]


In [26]:
pred_test = model.predict([X_test, X_categories_test])  # Pass both inputs as a list
pred_test = np.argmax(pred_test, axis=-1)  # Get the class with the highest probability
print(pred_test)

1/1 [==============================] - 0s 32ms/step
[[0 1 0 0 4 0 2 2 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [27]:
# post processing to get the order details in exr format
for i in range(len(test_data)):
    # Get original sequence lengths before padding
    original_length = len(test_data[i])
    
    # Extract predictions and true dev_labels for the original sequence
    pred_test_seq = pred_test[i][:original_length]